# DCF with ESG premium - Calculate the fair value of FAANGS and add premium for ESG scores

## Part 1
Import data from the CSV files with FAANG data and convert to pandas DataFrames.

In [2]:
#Import necessary modules
import pandas as pd
from pathlib import Path
%matplotlib inline

In [3]:
#Convert the FAANG data csv to a datframe.
faang_data_df = pd.read_csv(
    Path('./FAANG_DATA/stock_data.csv'),
    index_col="TICKER", 
)
faang_data_df.head()

,DEBT,CASH & EQUIVALENTS,MARGIN OF SAFETY,ESG SCORE
TICKER,,,,
META,16680,12681,45,1007.0
AAPL,119690,27502,45,886.0
AMZN,157560,37478,45,1428.0
NFLX,16910,5819,45,NaN
GOOG,28810,17936,45,1034.0


In [4]:
#Convert the individual FAANG company csv files to datframes.
meta_df = pd.read_csv(
    Path('./FAANG_DATA/meta.csv'),
    index_col="YEAR", 
)

aapl_df = pd.read_csv(
    Path('./FAANG_DATA/aapl.csv'),
    index_col="YEAR", 
)

amzn_df = pd.read_csv(
    Path('./FAANG_DATA/amzn.csv'),
    index_col="YEAR", 
)


goog_df = pd.read_csv(
    Path('./FAANG_DATA/goog.csv'),
    index_col="YEAR", 
)


In [5]:
#preview one of the dataframes.
amzn_df.head()

,2017,2018,2019,2020,2021
YEAR,,,,,
PERIOD,1.0000,2.0000,3.0000,4.0000,5.0000
FCF,6410.0000,17296.0000,21653.0000,25924.0000,-14726.0000
WACC,0.1028,0.1339,0.1131,0.0769,0.0664


## Calculate the DCFs for each company

### META

In [6]:
#Discount the cash flows for META from 2017-2020.
meta_growth_pv = meta_df.loc["FCF"]/((1+meta_df.loc["WACC"])**meta_df.loc["PERIOD"])
meta_growth_pv = meta_growth_pv.drop('2021')

In [7]:
#calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
meta_perpetuity_g = 0.05
meta_perpetuity_pv = (meta_df.at['FCF', '2021']*(1+meta_perpetuity_g))/(meta_df.at['WACC', '2021']- meta_perpetuity_g)
meta_wacc = meta_df.at['WACC', '2021']
dcf_meta_perpetuity_pv = meta_perpetuity_pv/(1+(meta_wacc))**5
dcf_meta_perpetuity_pv

558702.5089439774

In [8]:
#Preview the sum of all discounted cash flows
meta_dcf = meta_growth_pv.sum() + dcf_meta_perpetuity_pv
print(meta_dcf)
#Add esg premium, which will be a percent increase of the esg score * 0.01.
meta_esg = faang_data_df.loc["META", "ESG SCORE"]*0.01
print(meta_esg)
meta_mos = faang_data_df.loc["META", "MARGIN OF SAFETY"] * 0.01
print(meta_mos)

621502.3783077226
10.07
0.45


In [9]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
meta_dcf = meta_dcf - faang_data_df.loc["META", "DEBT"] + faang_data_df.loc["META", "CASH & EQUIVALENTS"]
meta_dcf = meta_dcf*(1+meta_esg)
print(meta_dcf)
meta_dcf_low = meta_dcf*(1-meta_mos)
print(meta_dcf_low)
meta_dcf_high = meta_dcf*(1+meta_mos)
print(meta_dcf_high)

6835762.397866489
3759669.3188265692
9911855.47690641


### AAPL

In [10]:
#Discount the cash flows for AAPL from 2017-2020.
aapl_growth_pv = aapl_df.loc["FCF"]/((1+aapl_df.loc["WACC"])**aapl_df.loc["PERIOD"])
aapl_growth_pv = aapl_growth_pv.drop('2021')

In [11]:
#calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
aapl_perpetuity_g = 0.05
aapl_perpetuity_pv = (aapl_df.at['FCF', '2020']*(1+aapl_perpetuity_g))/(aapl_df.at['WACC', '2021']- aapl_perpetuity_g)
aapl_wacc = aapl_df.at['WACC', '2021']
dcf_aapl_perpetuity_pv = aapl_perpetuity_pv/(1+(aapl_wacc))**5

In [12]:
#Preview the sum of all discounted cash flows
aapl_dcf = aapl_growth_pv.sum() + dcf_aapl_perpetuity_pv
aapl_dcf

1510660.4295669312

In [13]:
#Add esg premium, which will be a percent increase of the esg score * 0.01.
aapl_esg = faang_data_df.loc["AAPL", "ESG SCORE"]*0.01

aapl_mos = faang_data_df.loc["AAPL", "MARGIN OF SAFETY"] * 0.01

In [14]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
aapl_dcf = aapl_dcf - faang_data_df.loc["AAPL", "DEBT"] + faang_data_df.loc["AAPL", "CASH & EQUIVALENTS"]
aapl_dcf = aapl_dcf*(1+aapl_esg)
aapl_dcf_low = aapl_dcf*(1-aapl_mos)
aapl_dcf_high = aapl_dcf*(1+aapl_mos)
aapl_dcf

13986138.15552994

### AMZN

In [15]:
#Discount the cash flows for AAPL from 2017-2020.
amzn_growth_pv = amzn_df.loc["FCF"]/((1+amzn_df.loc["WACC"])**amzn_df.loc["PERIOD"])
amzn_growth_pv = amzn_growth_pv.drop('2021')
amzn_growth_pv

2017     5812.477330
2018    13452.288293
2019    15700.573909
2020    19275.271542
dtype: float64

In [16]:
amzn_positive_fcf = (0.2*amzn_df.at['FCF', '2021'] + 0.8*amzn_df.at['FCF', '2020'])
amzn_positive_fcf

17794.0

In [17]:
#Calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
amzn_perpetuity_g = 0.05
amzn_perpetuity_pv = (amzn_positive_fcf*(1+amzn_perpetuity_g))/(amzn_df.at['WACC', '2021']- amzn_perpetuity_g)
amzn_wacc = amzn_df.at['WACC', '2021']
dcf_amzn_perpetuity_pv = amzn_perpetuity_pv/(1+(amzn_wacc))**5
print(amzn_perpetuity_pv)

1139250.0000000002


In [18]:
#Preview the sum of all discounted cash flows
amzn_dcf = amzn_growth_pv.sum() + dcf_amzn_perpetuity_pv
amzn_dcf

880313.4734454678

In [19]:
#Add esg premium, which will be a percent increase of the esg score * 0.01.
amzn_esg = faang_data_df.loc["AMZN", "ESG SCORE"]*0.01

amzn_mos = faang_data_df.loc["AMZN", "MARGIN OF SAFETY"] * 0.01

In [20]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
amzn_dcf = amzn_dcf - faang_data_df.loc["AMZN", "DEBT"] + faang_data_df.loc["AMZN", "CASH & EQUIVALENTS"]
amzn_dcf = amzn_dcf*(1+amzn_esg)
amzn_dcf_low = amzn_dcf*(1-amzn_mos)
amzn_dcf_high = amzn_dcf*(1+amzn_mos)

amzn_dcf

11616336.91424675

### GOOG

In [21]:
#Discount the cash flows for META from 2017-2020.
goog_growth_pv = goog_df.loc["FCF"]/((1+goog_df.loc["WACC"])**goog_df.loc["PERIOD"])
goog_growth_pv = goog_growth_pv.drop('2021')

In [22]:
#calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
goog_perpetuity_g = 0.05
goog_perpetuity_pv = (goog_df.at['FCF', '2021']*(1+goog_perpetuity_g))/(goog_df.at['WACC', '2021']- goog_perpetuity_g)
goog_wacc = goog_df.at['WACC', '2021']
dcf_goog_perpetuity_pv = goog_perpetuity_pv/(1+(goog_wacc))**5
dcf_goog_perpetuity_pv

2080552.922620001

In [23]:
#Preview the sum of all discounted cash flows
goog_dcf = goog_growth_pv.sum() + dcf_goog_perpetuity_pv
print(goog_dcf)
#Add esg premium, which will be a percent increase of the esg score * 0.01.
goog_esg = faang_data_df.loc["GOOG", "ESG SCORE"]*0.01
print(goog_esg)
goog_mos = faang_data_df.loc["GOOG", "MARGIN OF SAFETY"] * 0.01
print(goog_mos)

2179049.5819982053
10.34
0.45


In [24]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
goog_dcf = goog_dcf - faang_data_df.loc["GOOG", "DEBT"] + faang_data_df.loc["GOOG", "CASH & EQUIVALENTS"]
goog_dcf = goog_dcf*(1+goog_esg)
print(goog_dcf)
goog_dcf_low = goog_dcf*(1-goog_mos)
print(goog_dcf_low)
goog_dcf_high =goog_dcf*(1+goog_mos)
print(goog_dcf_high)

24587111.099859647
13522911.104922807
35651311.094796486


## Print the results

### META

In [25]:
print(f"The fair value of META with an ESG premium of {meta_esg}% and a 45% margin of satefy should fall between:")
print(f"${meta_dcf_low:.2f} and ${meta_dcf_high:.2f}.")
print(f"Our mean fair value is ${meta_dcf:.2f}.")

The fair value of META with an ESG premium of 10.07% and a 45% margin of satefy should fall between:
$3759669.32 and $9911855.48.
Our mean fair value is $6835762.40.


### AAPL

In [26]:
print(f"The fair value of AAPL with an ESG premium of {aapl_esg}% and a 45% margin of satefy should fall between:")
print(f"${aapl_dcf_low:.2f} and ${aapl_dcf_high:.2f}.")
print(f"Our mean fair value is ${aapl_dcf:.2f}.")

The fair value of AAPL with an ESG premium of 8.86% and a 45% margin of satefy should fall between:
$7692375.99 and $20279900.33.
Our mean fair value is $13986138.16.


### AMZN

In [27]:
print(f"The fair value of AMZN with an ESG premium of {amzn_esg}% and a 45% margin of satefy should fall between:")
print(f"${amzn_dcf_low:.2f} and ${amzn_dcf_high:.2f}.")
print(f"Our mean fair value is ${amzn_dcf:.2f}.")

The fair value of AMZN with an ESG premium of 14.280000000000001% and a 45% margin of satefy should fall between:
$6388985.30 and $16843688.53.
Our mean fair value is $11616336.91.


### GOOG

In [28]:
print(f"The fair value of GOOG with an ESG premium of {goog_esg}% and a 45% margin of satefy should fall between:")
print(f"${goog_dcf_low:.2f} and ${goog_dcf_high:.2f}.")
print(f"Our mean fair value is ${goog_dcf:.2f}.")

The fair value of GOOG with an ESG premium of 10.34% and a 45% margin of satefy should fall between:
$13522911.10 and $35651311.09.
Our mean fair value is $24587111.10.


## Save the results

In [30]:
pd.set_option('display.float_format', '{:.2f}'.format)
data = [[meta_esg, aapl_esg, amzn_esg, goog_esg],
        [45, 45, 45, 45],
        [meta_dcf_low, aapl_dcf_low, amzn_dcf_low, goog_dcf_low],
        [meta_dcf_high, aapl_dcf_high, amzn_dcf_high, goog_dcf_high],
        [meta_dcf, aapl_dcf, amzn_dcf, goog_dcf]]
res_df = pd.DataFrame(
    data,
    index = ['esg_prem', 'margin_of_safety', 'dcf_low', 'dcf_high', 'dcf'],
    columns = [ 'META', 'AAPL', 'AMZN', 'GOOG']
)
res_df.to_csv('./FAANG_DATA/res.csv')